Импортируем библиотеку

In [1]:
%%time
from TSEnsemble import utils, nn, arima

CPU times: total: 4.16 s
Wall time: 4.58 s


Преобразуем .csv файл в DataFrame, состоящий из 1 столбца со значениями <br> временного ряда (column) и индекса, содержащего даты (index).

In [17]:
%%time
path = "datasets/GlobalTemperatures.csv"
dataset = utils.ts_from_csv(path, column = 1, index = 0)


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/GlobalTemperatures.csv'

Декомпозируем временной ряд на составляющие: тренд, сезонность, остатки.

In [3]:
%%time
utils.plot_decompose(path, period = 12)

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/GlobalTemperatures.csv'

Строим функцию автокорелляции для временного ряда, по ней можно увидеть сезонность ряда.

In [4]:
%%time
from statsmodels.graphics.tsaplots import plot_acf
utils.plot_acf(path, lags = 26)

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/GlobalTemperatures.csv'

Проверим ряд на стационарность, используя 2 теста: KPSS и ADF. <br> Также нарисуем график временного ряда, средних значений и стандартных<br> отклонений прошлых window значений (по умолчанию 2% датасета)

In [5]:
%%time
utils.isStationary(path, window = 24, plot = True)

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/GlobalTemperatures.csv'

Автоматически найдем наиболее оптимальные коэффициенты для SARIMA модели по критерию AIC и построим модель.

In [6]:
%%time
ar = arima.auto_arima(dataset,
                            method = 'stepwise',
                            season = 12,
                            max_p = 3,
                            max_q = 3,
                            max_Q = 3,
                            max_P = 3,
                            train_split = 0.95,
                            plot = True)

NameError: name 'dataset' is not defined

Построим CNN, LSTM, GRU, Transformer модели, пытаясь подобрать оптимальные параметры. 

In [7]:
%%time
cnn = nn.make_cnn(dataset,
                  look_back = 12,
                  hidden_layers = 1,
                  train_size = 0.95,
                  kernel_size = 2,
                  filters = 12,
                    epochs = 50,
                  batch_size = 64,
                  dilation_rate = 1,
                  dilation_mode = "additive")

NameError: name 'dataset' is not defined

In [8]:
%%time
lstm = nn.make_rnn(dataset,
                  look_back = 12,
                  units = 64,
                  hidden_layers = 1,
                  train_size = 0.95,
                  type = "LSTM",
                  epochs = 20,
                  batch_size = 16,
                  dropout = 0.0,
                  maxPlot = 100)

NameError: name 'dataset' is not defined

In [9]:
%%time
gru = nn.make_rnn(dataset,
                  look_back = 12,
                  hidden_layers = 1,
                  units = 64,
                  train_size = 0.95,
                  type = "GRU",
                  dropout = 0.0,
                  maxPlot = 100)

NameError: name 'dataset' is not defined

In [10]:
%%time
transformer = nn.make_transformer(dataset, 
                    look_back = 12, 
                    train_size = 0.95,
                    maxPlot = 100, 
                    batch_size = 32,
                    epochs = 30,
                    verbose = 0,
                    horizon = 1,
                    n_features = 1,
                    num_transformer_blocks = 4,
                    dropout = 0.25, 
                    head_size = 256, 
                    num_heads = 4, 
                    ff_dim = 4,
                    mlp_units=[128],
                    mlp_dropout=0.4)

NameError: name 'dataset' is not defined

Построим acl модель, используя параметры для моделей, подобранные ранее.

In [11]:
%%time
acl = nn.ACL()
acl = acl.fit(dataset,
              order = (3, 1, 0), 
              seasonal_order = (3, 1, 0, 12), 
              look_back = 12,
              train_size= 0.95, 
              filters = 64, 
              cnn_layers = 3, 
              dilation_mode = "multiplicative", 
              lstm_layers = 1,
              units = 64, 
              kernel_size = 2, 
              epochs = 40)


NameError: name 'dataset' is not defined

Сгенерируем ненатреннированные модели с уже подобранными параметрами, и создаздим ансамблевую модель, которая будет работать по принципу взвешенного среднего, основываясь на оценке ошибки MAPE.

In [12]:
%%time
from statsmodels.tsa.arima.model import ARIMA
ar = ARIMA(dataset, order = (3, 1, 0), seasonal_order = (3, 1, 0, 12))

transformer = nn.generate_transformer(
                    look_back = 12,
                    horizon = 1,
                    n_features = 1,
                    num_transformer_blocks = 4,
                    dropout = 0.25, 
                    head_size = 256, 
                    num_heads = 4, 
                    ff_dim = 4,
                    mlp_units=[128],
                    mlp_dropout=0.4)

lstm = nn.generate_rnn(look_back = 12,
                    hidden_layers = 1,
                    units = 64,
                    type = "LSTM",
                    dropout = 0.0)

cnn = nn.generate_cnn(look_back = 12,
                hidden_layers = 3,
                kernel_size = 2,
                filters = 64,
                dilation_rate = 1,
                dilation_mode = "multiplicative")
from TSEnsemble.ensemble import Ensemble
ensemble_model = Ensemble(models = [ar, cnn, lstm, transformer], regressor = 'wmean')
ensemble_model.fit(dataset, train_size = 0.95, look_back = 12, val_size = 0.2, train_models_size = 0.7, epochs = 20, batch_size = 16, metric = "rmse")
ensemble_model.eval(get="rmse")


NameError: name 'dataset' is not defined

In [13]:
%%time
from statsmodels.tsa.arima.model import ARIMA
ar = ARIMA(dataset, order = (3, 1, 0), seasonal_order = (3, 1, 0, 12))

transformer = nn.generate_transformer(
                    look_back = 12,
                    horizon = 1,
                    n_features = 1,
                    num_transformer_blocks = 4,
                    dropout = 0.25, 
                    head_size = 256, 
                    num_heads = 4, 
                    ff_dim = 4,
                    mlp_units=[128],
                    mlp_dropout=0.4)

lstm = nn.generate_rnn(look_back = 12,
                    hidden_layers = 1,
                    units = 64,
                    type = "LSTM",
                    dropout = 0.0)

cnn = nn.generate_cnn(look_back = 12,
                hidden_layers = 3,
                kernel_size = 2,
                filters = 64,
                dilation_rate = 1,
                dilation_mode = "multiplicative")
from TSEnsemble.ensemble import Ensemble
ensemble_model = Ensemble(models = [ar, cnn, lstm, transformer],
                          regressor = 'catboost',
                          regr_params ={'iterations': 20,                         
                                        'task_type' : "GPU",
                                        'devices' : '0:1',
                                        'depth': 10}
                         )
ensemble_model.fit(dataset, train_size = 0.95, look_back = 12, val_size = 0.2, train_models_size = 0.7, epochs = 20, batch_size = 16, metric = "rmse")
ensemble_model.eval(get="rmse")

NameError: name 'dataset' is not defined

In [14]:
ensemble_model = Ensemble(models = [ar, cnn, lstm, transformer],
                          regressor = 'catboost',
                          regr_params ={'iterations': 20,                         
                                        'task_type' : "GPU",
                                        'devices' : '0:1',
                                        'depth': 10}
                         )
ensemble_model.fit(dataset, fit_models = False, train_size = 0.95, look_back = 12, val_size = 0.2, train_models_size = 0.7, epochs = 20, batch_size = 16, metric = "rmse")
ensemble_model.eval(get="rmse")

NameError: name 'Ensemble' is not defined

In [ ]:
%%time
from statsmodels.tsa.arima.model import ARIMA
ar = ARIMA(dataset, order = (3, 1, 0), seasonal_order = (3, 1, 0, 12))
transformer = nn.generate_transformer(
                    look_back = 12,
                    horizon = 1,
                    n_features = 1,
                    num_transformer_blocks = 4,
                    dropout = 0.25, 
                    head_size = 256, 
                    num_heads = 4, 
                    ff_dim = 4,
                    mlp_units=[128],
                    mlp_dropout=0.4)


gru = nn.generate_rnn(
                  look_back = 12,
                  hidden_layers = 1,
                  units = 64,
                  type = "GRU",
                  dropout = 0.0)

cnn = nn.generate_cnn(look_back = 12,
                hidden_layers = 3,
                kernel_size = 2,
                filters = 64,
                dilation_rate = 1,
                dilation_mode = "multiplicative")

ensemble_model1 = Ensemble(models = [ar, cnn, gru, transformer], regressor = 'wmean')
ensemble_model1.fit(dataset, train_size = 0.95, look_back = 12, val_size = 0.2, train_models_size = 0.7, epochs = 20, batch_size = 16, metric = "rmse")
ensemble_model1.eval(get="rmse")

Также построим CNN LSTM модель.

In [ ]:
%%time
seq = nn.make_seq_model(dataset, 
            layers = ["cnn", "lstm"], 
            look_back = 12, 
            units = 64,
            filters = 64,
            horizon = 1, 
            dropout = 0.0, 
            n_features = 1, 
            kernel_size = 2, 
            dilation_rate = 1, 
            dilation_mode = "multiplicative",
            optimizer = "Adam",
            loss = "mae",
            train_size = 0.95, 
            plot = True,
            batch_size = 16,
            epochs = 30,
            maxPlot = 100, 
            fig_size = (15, 5))

Построим предсказания на следующие n значений используя некоторые из моделей.

In [ ]:
%%time
ensemble_model.forecast(dataset, 12, fig_size = (5,5))

In [ ]:
%%time
# from statsmodels.tsa.arima.model import ARIMA
# ar = ARIMA(dataset, order = (2, 0, 1), seasonal_order = (2, 0, 1, 23))
utils.model_forecast(ar, dataset, 12, fig_size = (5,5))